In [ ]:
'''
Denote the iteration before doing any conversion or filteration. 
Filtering the second time or converting with different function can lead to data alignment issues
'''

In [8]:
import stltovoxel as stv
import numpy as np
import nrrd
from stl import mesh
import glob
import os
from scipy import ndimage

In [2]:
def checkmeshparam(file_path, resolution = 100):
    mesh_obj = mesh.Mesh.from_file(file_path)
    org_mesh = np.hstack((mesh_obj.v0[:, np.newaxis], mesh_obj.v1[:, np.newaxis], mesh_obj.v2[:, np.newaxis]))
    obj = []
    obj.append(org_mesh)
    vol, scale, shift = stv.convert_meshes(obj, resolution, None, False)
    vol = vol.astype(np.uint8)
    return vol, vol.shape, scale, shift    

In [3]:
def convertmesh(file_path):
    vol, size, sfa, shift = checkmeshparam(file_path)
    sf = np.mean(sfa)
    if sf <= 0.975 or sf >= 1.025:
        resolution = int(np.floor(100/(np.mean(sf))))
        print(f"___________Rescaled to {resolution} Resolution___________") 
        vol, size, sfa, shift = checkmeshparam(file_path, resolution)
        sf = np.mean(sfa)
        
    return vol, size, shift, sf

In [4]:
def createnrrd(vol, size, shift, file_path):
    origin = (shift[0],shift[1],shift[2])
    trans_vol = np.rot90(np.flip(vol, axis = 2), k =1, axes = (0,2))
    header = {
    'type' : 'u1',
    'dimension' : '3',
    'space': 'left-posterior-superior',
    'sizes': size,
    'space directions': [(1, 0, 0), (0, 1, 0), (0, 0, 1)],
    'kinds': ['domain', 'domain', 'domain'],
    'encoding': 'gzip',
    'space origin': origin
    }
    nrrd.write(file_path.replace(".stl",".nrrd"), trans_vol, header) 
    

In [5]:
def convert_to_nrrd(file_path):
    file_name = file_path.split('\\')[-1].split('.')[0]
    vol, size, shift, scaled = convertmesh(file_path)
    createnrrd(vol, size, shift,file_path)
    print(f"{file_name} created with {scaled} acc")
    pass

In [ ]:
skulls_directory = '.\Skulls'
for folder_name in os.listdir(skulls_directory):
    for file in os.listdir(os.path.join(skulls_directory,folder_name)):
        if 'implant' in file:
            convert_to_nrrd(os.path.join(skulls_directory,folder_name,file)) 


In [35]:
def nrrdfilter(file_path):
    filename = file_name = file_path.split('\\')[-1].split('.')[0]
    skull_vol,header = nrrd.read(file_path)
    header = dict(header)
    # Apply opening (erosion followed by dilation)
    opened_data = (ndimage.binary_opening(skull_vol, structure=np.ones((2, 2, 2)), iterations=1)).astype('uint8')
    nrrd.write(file_path, np.flip(opened_data, axis=(0,1)), header)
    print(f"{file_name} filtered and replaced")

In [28]:
lis = []

In [29]:
skulls_directory = '.\Skulls'
for folder_name in os.listdir(skulls_directory):
    for file in os.listdir(os.path.join(skulls_directory,folder_name)):
        if 'implant' not in file and '.stl' not in file:
            lis.append(os.path.join(skulls_directory,folder_name,file)) #directly run thr filter function here 


In [ ]:
lis = lis[8:] #first 8 files already converted in trials

In [ ]:
for item in lis:
    nrrdfilter(item)